In [ ]:
import polars as pl
from datetime import datetime,timedelta
import get_and_store as gas
import polars_indicators as pi
from polars_indicators.indicators import IndicatorResult
from importlib import reload
DATE_COLUNN = "Date"

In [ ]:
reload(pi)
reload(pi.strategies.buy_x_week_low)
reload(pi.strategies)
reload(pi.strategies.buy_second_dip_in_uptrend)
reload(pi.summarize.trades)
reload(pi.indicators)

In [ ]:
symbols = ['A','AA']

In [ ]:
#df = gas.get_symbols_df(symbols)

In [ ]:
try:
    df = saved_df.clone()
except NameError:
        df = gas.get_all_saved_data_df()
        df = df.filter(pl.col(pi.indicators.DATE_COLUMN) >= datetime(2020, 1, 1))
        df = df.collect()

In [ ]:
saved_df = df.clone()

In [ ]:
sma100 = pi.indicators.simple_moving_average(df, 100)
df = sma100.df
slope_up = pi.indicators.slope_up(df, column=sma100.column)
df = slope_up.df.with_columns(pl.col(slope_up.column).shift()) #shifting so we use previous bar

In [ ]:
df = df.with_columns(pl.col(pi.indicators.DATE_COLUMN).cast(pl.Datetime))
df

In [ ]:
#filter out average price too low
df = df.filter(pl.col(pi.indicators.CLOSE_COLUMN).mean().over(pi.indicators.SYMBOL_COLUMN) > 3)

# filter out too many 0 volume bars. This gets rid of a lot of garbage
df = df.filter((pl.col(pi.indicators.VOLUME_COLUMN) == 0).sum().over(pi.indicators.SYMBOL_COLUMN) < 3)

# minimum amount of volume
df = df.filter(pl.col(pi.indicators.VOLUME_COLUMN).mean().over(pi.indicators.SYMBOL_COLUMN) > 100000)


In [ ]:
lookback_bars = 25
offset_bars = 7
strat_result = pi.strategies.buy_second_dip_in_uptrend.strategy(df, lookback_bars, offset_bars, 1, 2, 3)

In [ ]:
trades = pi.summarize.trades.Trades.from_strategy_result(strat_result, [slope_up.column])

In [ ]:
trades.summarize_strategy()

In [ ]:
trades.df.filter(pl.col("Gain/Loss%") < -25.54).sort(pl.col("Gain/Loss%"))

In [ ]:
trades.df.filter(pl.col("Highest%") > 586.16)

In [ ]:
tf = trades.df.groupby(pi.indicators.SYMBOL_COLUMN).agg(pl.col(pi.indicators.SYMBOL_COLUMN).count().alias("trades")).sort("trades", descending=True)
tf

In [ ]:
tester = strat_result.df.with_row_count("row").filter(pl.col("row").is_between(820,831)).select("Date","Open","High","Low","Close","Volume","Symbol")